<a href="https://colab.research.google.com/github/Shiladitya56/Machine-Learning/blob/main/SEM2/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [225]:
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import random
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

In [226]:
def r2(actual: np.ndarray, predicted: np.ndarray):
  """ R2 Score """
  return r2_score(actual, predicted)
def adjr2(model, x: np.ndarray, y: np.ndarray):
  """ Adjusted R2 Score """
  return 1 - ( 1-model.score(x, y) ) * ( len(y) - 1 ) / ( len(y) - x.shape[1] - 1 )

In [227]:
df = pd.read_csv('/content/drive/MyDrive/ipynb/sem2/datasets/prog5.csv')
df.head()

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [228]:
df.isnull().sum()

Unnamed: 0     0
Category       0
Age            0
Sex            0
ALB            2
ALP           20
ALT            3
AST            2
BIL            0
CHE            0
CHOL          11
CREA           1
GGT            0
PROT           1
dtype: int64

In [229]:
df.dropna(inplace=True)

In [230]:
x = df['CREA']
y = df['PROT']

In [231]:
x = (x - x.mean())/x.std()
y = (y - y.mean())/y.std()

In [232]:
def permulit(x, y, test_size=0.2, random_seed=None):
  '''
    Split the dataset into training and testing sets.
    Parameters:
    - x: Features (list, array, or DataFrame)
    - y: Labels (list, array, or Series)
    - test_size: The proportion of the dataset to include in the test split (default is 0.2)
    - random_seed: Seed for random number generation to ensure reproducibility (default is None)

    Returns:
    - x_train, y_train, x_test, y_test
  '''
  assert len(x) == len(y),\
  'Input arrays x and y must have the same length (:'
  assert 0 < test_size < 1,\
  'test_size percentage should be a float between 0 and 1.'
  if random_seed:
    random.seed(random_seed)
  combined_data = list(zip(x, y))
  random.shuffle(combined_data)
  x_shuffled, y_shuffled = zip(*combined_data)
  split_index = int(len(x) * (1 - test_size))
  x_train, x_test = x_shuffled[:split_index], x_shuffled[split_index:]
  y_train, y_test = y_shuffled[:split_index], y_shuffled[split_index:]
  return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

In [233]:
pd.Series([vif(df[['ALB','ALP','ALT','AST','BIL','CHE','CHOL','CREA','GGT']].values, i) for i in range(df[['ALB','ALP','ALT','AST','BIL','CHE','CHOL','CREA','GGT']].shape[1])], index=df[['ALB','ALP','ALT','AST','BIL','CHE','CHOL','CREA','GGT']].columns)

ALB     31.162740
ALP     10.024253
ALT      3.184002
AST      2.938453
BIL      1.678633
CHE     22.397373
CHOL    27.238712
CREA     3.656226
GGT      2.542260
dtype: float64

In [234]:
x_train, y_train, x_test, y_test = permulit(x,y,test_size=0.2)

In [235]:
model = linear_model.LinearRegression()
model.fit(x_train.reshape(-1,1), y_train.reshape(-1,1))

LinearRegression()

In [236]:
y_pred = model.predict(x_test.reshape(-1,1))

In [237]:
print('The Mean Squared Error using Polynomial Regression is: ', mse(y_test, y_pred))
print('The R-squared Error using Polynomial Regression is: ', r2(y_test, y_pred))
print('The Adjusted R-squared Error using Polynomial Regression is: ', adjr2(model, y_test.reshape(-1,1), y_pred))

The Mean Squared Error using Polynomial Regression is:  0.9569751117261746
The R-squared Error using Polynomial Regression is:  -0.015959483737699065
The Adjusted R-squared Error using Polynomial Regression is:  -6.985970500195253


In [238]:
print('The slope is : ', model.coef_)
print('The y-intercept is : ', model.intercept_)

The slope is :  [[-0.04813851]]
The y-intercept is :  [-0.01820825]


In [239]:
model = LinearRegression()
p = PolynomialFeatures(degree=3, include_bias=True)

In [240]:
x = df['ALB']
y = df['PROT']
x = (x - x.mean())/x.std()
y = (y - y.mean())/y.std()
x_train, y_train, x_test, y_test = permulit(x,y,test_size=0.2)

In [241]:
def poly():
  for i in range(2,5,1):
    p = PolynomialFeatures(degree=i*2, include_bias=True)
    x_trainP = p.fit_transform(x_train.reshape(-1,1))
    x_testP = p.transform(x_test.reshape(-1,1))
    model.fit(x_trainP, y_train)
    y_pred = model.predict(x_testP)
    print(f'The Mean Squared Error using Polynomial Regression with degree {i*2} is: ', mse(y_test, y_pred))
    print('The slope is : ', model.coef_)
    print('The y-intercept is : ', model.intercept_)

In [242]:
poly()

The Mean Squared Error using Polynomial Regression with degree 4 is:  0.5728788428700268
The slope is :  [ 0.          0.53800187 -0.07858299  0.00319046 -0.00085719]
The y-intercept is :  0.07606844851833572
The Mean Squared Error using Polynomial Regression with degree 6 is:  0.5573512875481054
The slope is :  [ 0.00000000e+00  5.61713489e-01  8.08949602e-02  2.63020891e-03
 -2.45313839e-02 -9.76696713e-04  5.53132298e-04]
The y-intercept is :  0.007735085602030432
The Mean Squared Error using Polynomial Regression with degree 8 is:  0.5555702780817107
The slope is :  [ 0.00000000e+00  6.13902441e-01  8.18240338e-02 -4.01500122e-02
 -2.54284738e-02  3.94075057e-03  5.33632054e-04 -1.25381397e-04
  6.88730247e-06]
The y-intercept is :  0.006681522803544716


In [243]:
ridge = Ridge()
parameters={'alpha':[0.001,0.01,1,5,10,20,30,35,40,45,50,55,100,150]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(x_train.reshape(-1,1),y_train.reshape(-1,1))
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 20}
-0.7163966526446914


In [244]:
y_pred = ridge_regressor.predict(x_test.reshape(-1,1))
print(f'The Mean Squared Error using Ridge Regression is: ', mse(y_test.reshape(-1,1), y_pred))
print(f'The R-squared error using Ridge Regression is: ', r2(y_test, y_pred))
print(f'The Adjusted R-squared error using Ridge Regression is: ', adjr2(ridge_regressor, y_test.reshape(-1,1), y_pred))

The Mean Squared Error using Ridge Regression is:  0.5980549722262198
The R-squared error using Ridge Regression is:  0.3110924857448517
The Adjusted R-squared error using Ridge Regression is:  -0.1950941596768836


In [245]:
lasso = Lasso()
parameters={'alpha':[0.001,0.01,1,5,10,20,30,35,40,45,50,55,100]}
lasso_regressor=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=5)
lasso_regressor.fit(x_train.reshape(-1,1),y_train.reshape(-1,1))
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 20}
-0.7163966526446914


In [246]:
y_pred = lasso_regressor.predict(x_test.reshape(-1,1))
print(f'The Mean Squared Error using Lasso Regression is: ', mse(y_test.reshape(-1,1), y_pred))
print(f'The R-squared error using Lasso Regression is: ', r2(y_test, y_pred))
print(f'The Adjusted R-squared error using Lasso Regression is: ', adjr2(lasso_regressor, y_test.reshape(-1,1), y_pred))

The Mean Squared Error using Lasso Regression is:  0.5959851291025009
The R-squared error using Lasso Regression is:  0.3134767656981666
The Adjusted R-squared error using Lasso Regression is:  -0.20353822853991677
